UNIVERSIDADE ESTADUAL DO CEARÁ \
MESTRADO ACADÊMICO EM CIÊNCIA DA COMPUTAÇÃO \
MINERAÇÃO MASSIVA DE DADOS

Daniel Gleison Moreira Lira \
daniel.gleison@aluno.uece.br

---
# Aplicativo de diagnóstico de diabetes utilizando aprendizado de máquina e processamento distribuído
---

In [1]:
import findspark
findspark.init()

In [5]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.linalg import Vectors

import pandas as pd

In [3]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("PredictionDiabetesApp") \
        .getOrCreate()

In [7]:
#Carregando o modelo de classificação Random Foreset
Model_RF_PredictionFraud = RandomForestClassificationModel.load('model_rf')

In [73]:
In1 =  input("Idade:")
In2 =  input("Gênero [M/F]:")
In3 =  input("Poliúria [S/N]:")
In4 =  input("Polidipsia [S/N]:")
In5 =  input("Perda repentina de peso [S/N]:")
In6 =  input("Fraqueza [S/N]:")
In7 =  input("Polifagia [S/N]:")
In8 =  input("Tordo genital [S/N]:")
In9 =  input("Embaçamento visual [S/N]:")
In10 =  input("Coceira [S/N]:")
In11 = input("Irritabilidade [S/N]:")
In12 = input("Demora de cura [S/N]:")
In13 = input("Paresia parcial [S/N]:")
In14 = input("Rigidez muscular [S/N]:")
In15 = input("Alopecia [S/N]:")
In16 = input("Obesidade [S/N]:")

Idade: 50
Gênero [M/F]: M
Poliúria [S/N]: S
Polidipsia [S/N]: S
Perda repentina de peso [S/N]: N
Fraqueza [S/N]: N
Polifagia [S/N]: S
Tordo genital [S/N]: N
Embaçamento visual [S/N]: S
Coceira [S/N]: S
Irritabilidade [S/N]: S
Demora de cura [S/N]: S
Paresia parcial [S/N]: N
Rigidez muscular [S/N]: S
Alopecia [S/N]: N
Obesidade [S/N]: S


In [74]:
data = [In1,In2,In3,In4,In5,In6,In7,In8,In9,In10,In11,In12,In13,In14,In15,In16]
column_names = ["Idade","Genero","Poliuria","Polidipsia","Perda_Peso","Fraqueza","Polifagia","Tordo_genital",\
                "Embacamento_visual","Coceira","Irritabilidade",\
               "Demora_cura","Paresia_parcial","Rigidez_muscular","Alopecia","Obesidade"]
df = pd.DataFrame(data, column_names)

In [75]:
#Transformação dos atributos em [0/1]

if  df[0][1] == 'M' or df[0][1] == 'm': df[0][1] = 1 
elif df[0][1]  == 'F' or df[0][1]  == 'f': df[0][1] = 0
else: df[0][1]  = 0
        
for x in range(2,16):
    if   df[0][x] == 'S' or df[0][x] == 's': df[0][x] = 1 
    elif df[0][x]  == 'N' or df[0][x]  == 'n': df[0][x] = 0
    else: df[0][x]  = 0

In [76]:
list = [df[0][0],df[0][1],df[0][2],df[0][3],df[0][4],df[0][5],\
        df[0][6],df[0][7],df[0][8],df[0][9],df[0][10],\
        df[0][11],df[0][12],df[0][13],df[0][14],df[0][15]]

In [77]:
features = Vectors.dense(list)

In [112]:
df_app = spark.createDataFrame(
    [('',\
      Vectors.dense(features))],
    ['', 'features'])

In [122]:
df_app = spark.createDataFrame ([(Vectors.dense(features))],[features'])

SyntaxError: EOL while scanning string literal (<ipython-input-122-2ca3873b9fed>, line 1)

In [113]:
resultado = Model_RF_PredictionFraud.transform(df_app)
df_predicao = resultado.select('features','rawPrediction','probability','prediction')

In [114]:
df_predicao = df_predicao.toPandas()
df_predicao

,features,rawPrediction,probability,prediction
0,"[50.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0,...","[3.8772510278481045, 46.122748972151896]","[0.0775450205569621, 0.9224549794430379]",1.0


In [115]:
vl_predicao = df_predicao.iloc[0][3]
vl_prob_negativo = df_predicao.iloc[0][2][0]
vl_prob_positivo = df_predicao.iloc[0][2][1]

In [116]:
print('RESULTADO DA PREDIÇÃO:')
if vl_predicao == 1: print('DIAGNÓSTICO POSITIVO')
if vl_predicao == 0: print('DIAGNÓSTICO NEGATIVO')
print("")
print('PROBABILIDADE:')
if vl_prob_negativo >= vl_prob_positivo: print(vl_prob_negativo*100,'%')
if vl_prob_negativo < vl_prob_positivo: print(vl_prob_positivo*100, '%')

RESULTADO DA PREDIÇÃO:
DIAGNÓSTICO POSITIVO

PROBABILIDADE:
92.24549794430379 %


---

In [14]:
spark.stop()